# Phyling API

To use this notebook, you need to create a `.env` file with:

```env
PHYLING_MAIL=your_email
PHYLING_PASSWORD=your_password
```

In [1]:
import json
import time
import logging
import os
from dotenv import load_dotenv

from phyling.api import PhylingAPI
from phyling.api import PhylingRealtime

load_dotenv()


True

# API connection

In [ ]:
api = PhylingAPI(
    mail=os.getenv("PHYLING_MAIL"),
    password=os.getenv("PHYLING_PASSWORD"),
    # url="https://api.app.phyling.fr/"
    # url="https://staging.api.cl.phyling.fr/"
    url="http://localhost:5001/"
)
print(api)

realtime = PhylingRealtime(api)
print(realtime)

PhylingAPI(mail=tim.nicolas@phyling.fr, baseurl=http://localhost:5001) -> Connected
PhylingRealtime(mail=tim.nicolas@phyling.fr, baseurl=http://localhost:5001) -> Connected


In [58]:
realtime.autoUpdateDeviceList(True)

In [59]:
print(realtime.getDeviceList())

[{'is_connected': True, 'name': 'Maxi 300', 'number': 10200300, 'state': 'idle'}]


In [60]:
number = realtime.getDeviceList()[0]["number"]
realtime.autoUpdateDeviceStatus(number=number, enabled=True)
realtime.autoUpdateDeviceIndicator(number=number, enabled=True)
realtime.autoUpdateDeviceData(number=number, enabled=True)

In [62]:
print(realtime.getDeviceStatus(number=number))

{'battery': 100, 'client': {'id': 2, 'name': 'phyling'}, 'deviceRecId': 193, 'epoch': 1759394156.0, 'features': {'calib': {'cmd_get': 'v1.board.calib.get', 'cmd_set': 'v1.board.calib.set'}, 'config': {'cmd_get': 'v1.board.config.get', 'cmd_set': 'v1.board.config.set'}, 'device_info': {'cmd_get': 'v1.board.device_info.get', 'cmd_set': 'v1.board.device_info.set'}, 'firmware_update': {'cmd': 'v1.board.firmware_update'}, 'logs': {'cmd_get': 'v1.logs.get', 'cmd_list': 'v1.logs.list'}, 'record': {'cmd_realtime': 'v1.record.realtime.set', 'cmd_start': 'v1.record.rec.start', 'cmd_stop': 'v1.record.rec.stop'}, 'restart': {'cmd': 'v1.board.restart'}, 'scenario': {'cmd_realtime': 'v1.record.realtime.set', 'cmd_start': 'v1.record.scenario.start', 'cmd_stop': 'v1.record.scenario.stop'}, 'shutdown': {'cmd': 'v1.board.shutdown'}, 'time_set': {'cmd': 'v1.board.time.set'}, 'wifi': {'cmd_get': 'v1.board.wifi.get', 'cmd_set': 'v1.board.wifi.set'}}, 'group': {'id': 1, 'name': 'phyling'}, 'is_connected': T

In [9]:
print(realtime.getDeviceStatus(number=number)["state"])

scenario


In [66]:
print(realtime.getDeviceIndicator(number=number))

{'number': 10200300, 'recTime': 178.123, 'indicators': {'acc_x': {'x': 177.842491, 'y': 0.2968113739043474}}}


In [41]:
lastRtData = realtime.getDeviceData(number=number)
print(lastRtData)
print(lastRtData["recTime"])
print(lastRtData["data"]["imu"]["T"][-1])

{'number': 10200300, 'recTime': 1225.612536430359, 'recTimeSinceEpoch': 1759392302.368264, 'data': {'imu': {'acc_x': [0.296811, 0.311173, 0.311173, 0.320748, 0.320748, 0.311173, 0.320748], 'acc_y': [-0.071809, -0.114895, -0.071809, -0.100533, -0.081384, -0.119682, -0.081384], 'acc_z': [9.727753, 9.727753, 9.756477, 9.785201, 9.718179, 9.785201, 9.737328], 'gyro_x': [-0.282938, -0.212938, -0.282938, -0.352938, -0.212938, -0.282938, -0.212938], 'gyro_y': [0.171232, 0.101232, 0.101232, 0.171232, 0.241232, 0.171232, 0.101232], 'gyro_z': [0.571374, 0.571374, 0.571374, 0.501374, 0.571374, 0.571374, 0.571374], 'T': [1225.339, 1225.359, 1225.38, 1225.4, 1225.42, 1225.44, 1225.46], 'acc': [9.732545, 9.733407, 9.761702, 9.790972, 9.723811, 9.790879, 9.742949]}}, 'selections': []}
1225.612536430359
1225.46


In [46]:
round(1759393464.240169, 4)

1759393464.2402

In [70]:
# while loop on device data
ONLY_FIRST = True
lastRecTime = 0
while True:
    status = realtime.getDeviceStatus(number=number)
    rtData = realtime.getDeviceData(number=number)
    now = time.time()
    if rtData["recTime"] <= lastRecTime:
        continue
    lastRecTime = rtData["recTime"]
    dataStr = f"[{rtData['recTime']:6.3f}] "
    for modname, modcontent in rtData.get("data", {}).items():
        latency = now - status['startRecTimeSinceEpoch'] - rtData["data"][modname]["T"][-1]
        dataStr +=  f"latency: {int(latency * 1000):4d}ms"
        for key, value in modcontent.items():
            if key in ["T"]:
                continue
            if len(dataStr) > 0:
                dataStr += " | "
            dataStr += f"{modname}.{key}: {value[-1]:.2f}"
            if ONLY_FIRST:
                break
    if len(dataStr) > 0:
        print(dataStr)
    time.sleep(0.01)

[411.612] latency:  266ms | imu.acc_x: 0.34
[411.722] latency:  207ms | imu.acc_x: 0.29
[411.891] latency:  232ms | imu.acc_x: 0.32
[412.000] latency:  223ms | imu.acc_x: 0.29
[412.110] latency:  180ms | imu.acc_x: 0.29
[412.278] latency:  254ms | imu.acc_x: 0.31
[412.379] latency:  195ms | imu.acc_x: 0.27
[412.491] latency:  173ms | imu.acc_x: 0.28
[412.601] latency:  178ms | imu.acc_x: 0.32
[412.775] latency:  260ms | imu.acc_x: 0.29
[412.888] latency:  211ms | imu.acc_x: 0.28
[412.999] latency:  201ms | imu.acc_x: 0.31
[413.171] latency:  260ms | imu.acc_x: 0.34
[413.281] latency:  214ms | imu.acc_x: 0.33
[413.396] latency:  194ms | imu.acc_x: 0.28
[413.573] latency:  239ms | imu.acc_x: 0.31
[413.678] latency:  217ms | imu.acc_x: 0.30
[413.802] latency:  190ms | imu.acc_x: 0.30
[413.979] latency:  251ms | imu.acc_x: 0.02
[414.081] latency:  223ms | imu.acc_x: 0.31
[414.184] latency:  217ms | imu.acc_x: 1.23
[414.357] latency:  226ms | imu.acc_x: -2.75
[414.462] latency:  221ms | imu

KeyboardInterrupt: 

In [69]:
# while loop on device indicators
# {'number': 10200300, 'recTime': 178.123, 'indicators': {'acc_x': {'x': 177.842491, 'y': 0.2968113739043474}}}
ONLY_FIRST = False
lastRecTime = 0
while True:
    status = realtime.getDeviceStatus(number=number)
    rtIndicators = realtime.getDeviceIndicator(number=number)
    now = time.time()
    if rtIndicators["recTime"] <= lastRecTime:
        continue
    lastRecTime = rtIndicators["recTime"]
    dataStr = f"[{rtIndicators['recTime']:6.3f}] "
    for indname, indcontent in rtIndicators.get("indicators", {}).items():
        latency = now - status['startRecTimeSinceEpoch'] - rtIndicators["indicators"][indname]["x"]
        dataStr +=  f"latency: {int(latency * 1000):4d}ms {indname}: {indcontent['y']:.2f}"
        if ONLY_FIRST:
            break
    if len(dataStr) > 0:
        print(dataStr)
    time.sleep(0.01)

[382.254] latency:  899ms acc_x: 0.29
[382.840] latency:  370ms acc_x: 0.31
[383.373] latency:  337ms acc_x: 0.31
[384.025] latency:  338ms acc_x: 0.30
[384.564] latency:  337ms acc_x: 0.32
[385.082] latency:  347ms acc_x: 0.32
[385.730] latency:  339ms acc_x: 0.51
[386.391] latency:  364ms acc_x: -9.34
[386.999] latency:  298ms acc_x: -9.11
[387.524] latency:  301ms acc_x: -8.89
[388.079] latency:  315ms acc_x: -9.15
[388.593] latency:  332ms acc_x: -1.32
[389.138] latency:  329ms acc_x: -1.35
[389.796] latency:  339ms acc_x: -1.33
[390.353] latency:  368ms acc_x: -1.26
[390.860] latency:  349ms acc_x: -1.20
[391.473] latency:  320ms acc_x: -1.02
[392.022] latency:  310ms acc_x: -0.82
[392.555] latency:  323ms acc_x: -0.63
[393.060] latency:  306ms acc_x: -3.08
[393.606] latency:  325ms acc_x: -8.37
[394.395] latency:  847ms acc_x: -8.73
[394.901] latency:  336ms acc_x: -7.11
[395.409] latency:  322ms acc_x: 9.62
[395.949] latency:  329ms acc_x: 0.26
[396.613] latency:  331ms acc_x: 0

KeyboardInterrupt: 

In [ ]:
realtime.executeRPC(
    number=number,
    method="v1.record.rec.stop",
    params={},
    wait_for_result=False
)